In [2]:
import pandas as pd
import math
from itertools import combinations
from itertools import permutations

<div style="display: flex; gap: 10px;">
    <img src="stats1.png" alt="image1" width="600">
    <img src="stats2.png" alt="image2" width="600"
</div>


In [3]:
all_chars = ["HP", "HP%", "АТК", "АТК%", "ДЕФ", "ДЕФ%", "ВЭ%", "МС", "КУ", "КШ"]
mainStatArt1Weights = {
    "HP": 1,
}
mainStatArt2Weights = {
    "АТК": 1,
}
mainStatArt3Weights = {
    "HP": 8,      #26.(6)
    "АТК%": 8, #26.(6)
    "ДЕФ": 8,   #26.(6)
    "ВЭ%": 3,   #10
    "МС": 3      #10
}
mainStatArt4Weights = {
    "HP": 1925,
    "АТК%": 1925,
    "ДЕФ": 1900,
    "ФИЗ%": 500,
    "АМЕНО%": 500,
    "ПИРО%": 500,
    "ГИДРО%": 500,
    "ГЕО%": 500,
    "ДЕНДРО%": 500,
    "КРИО%": 500,
    "ЭЛЕКТРО%": 500,
    "МС": 250   
}
mainStatArt5Weights = {
    "HP": 2200,
    "АТК%": 2200,
    "ДЕФ": 2200,
    "КУ": 1000,
    "КШ": 1000,
    "МС": 400,
    "ЛЕЧ": 1000   
}

In [4]:
extraStatsWeights = {
    "HP": 6,
    "HP%": 4,
    "АТК": 6,
    "АТК%": 4,
    "ДЕФ": 6,
    "ДЕФ%": 4,
    "ВЭ%": 4,
    "МС": 4,
    "КУ": 3,
    "КШ": 3
}

In [5]:
# Предполагаем равновероятное выпадение любого слота из 5 слотов:
slot_weights = {
    "Цветок": 1,
    "Перо": 1,
    "Часы": 1,
    "Кубок": 1,
    "Шапка": 1
}

slot_to_main_stats = {
    "Цветок": mainStatArt1Weights,
    "Перо": mainStatArt2Weights,
    "Часы": mainStatArt3Weights,
    "Кубок": mainStatArt4Weights,
    "Шапка": mainStatArt5Weights
}

In [6]:
# Функция для получения вероятности конкретного основного стата в данном слоте
def get_main_stat_probability(slot, main_stat):
    weights = slot_to_main_stats[slot]
    total = sum(weights.values())
    return weights.get(main_stat, 0) / total if total > 0 else 0

In [7]:
def get_substats_probability_as_set(main_stat, desired_substats):
    # Проверим входные данные
    if len(desired_substats) != 3:
        return 0.0

    # Формируем пул сабстатов без основного
    available_substats = {k: v for k, v in extraStatsWeights.items()}
    if main_stat in available_substats:
        del available_substats[main_stat]

    # Проверим, что все желаемые сабстаты доступны
    for s in desired_substats:
        if s not in available_substats:
            return 0.0

    # Суммируем вероятность по всем перестановкам
    total_probability = 0.0
    str = "Вероятности сабстатов:"
    for perm in permutations(desired_substats):
        # Пошаговый расчёт вероятности для данной перестановки:
        # 1-й сабстат:
        total_weight_1 = sum(available_substats.values())
        w1 = available_substats[perm[0]]
        p1 = w1 / total_weight_1

        # 2-й сабстат:
        pool_after_first = {k: v for k, v in available_substats.items() if k != perm[0]}
        total_weight_2 = sum(pool_after_first.values())
        w2 = pool_after_first[perm[1]]
        p2 = w2 / total_weight_2

        # 3-й сабстат:
        pool_after_second = {k: v for k, v in pool_after_first.items() if k != perm[1]}
        total_weight_3 = sum(pool_after_second.values())
        w3 = pool_after_second[perm[2]]
        p3 = w3 / total_weight_3

        #str += f"\n{perm[0]}: {p1:.4f}, {perm[1]}: {p2:.4f}, {perm[2]}: {p3:.4f} => {p1 * p2 * p3:.10f}"
        str += f"\n{perm[0]}: {w1} / {total_weight_1} = {p1:.4f}, {perm[1]}: {w2} / {total_weight_2} = {p2:.4f}, {perm[2]}: {w3} / {total_weight_3} = {p3:.4f} => {p1 * p2 * p3:.10f}"
        total_probability += p1 * p2 * p3
    
    print(str)

    # Теперь total_probability - это вероятность получить эти три сабстата без учёта порядка.
    return total_probability

In [8]:

# Итоговая функция для рассчёта вероятности получить артефакт с заданным слотом, основным статом и набором сабстатов:
def get_artifact_probability(slot, main_stat, substats):
    # Вероятность слота:
    total_slot_weight = sum(slot_weights.values())
    p_slot = slot_weights[slot] / total_slot_weight
    print(f"Вероятность слота: {p_slot}")
    # Вероятность основного стата:
    p_main = get_main_stat_probability(slot, main_stat)
    print(f"Вероятность основного стата: {p_main}")
    # Вероятность сабстатов:
    p_sub = get_substats_probability_as_set(main_stat, substats)
    print(f"Суммарная вероятность сабстатов: {p_sub}\n")

    return p_slot * p_main * p_sub

In [9]:
# Пример: хотим получить артефакт из слота кубок с основным статом "ПИРО%" и сабстатами ["КШ", "КУ", "АТК%"]
slot = "Кубок"
desired_main_stat = "ПИРО%"
desired_substats = ["КШ", "КУ", "АТК%"]

print(f"Рассчёт вероятности получить артефакт с заданными параметрами:")
print(f"Слот: {slot}, основной стат: {desired_main_stat}, сабстаты: {desired_substats}\n")

p = get_artifact_probability(slot, desired_main_stat, desired_substats)
print(f"Вероятность получить {slot} с {desired_main_stat} и сабстатами {desired_substats}: {p:12f}")

# Оценка времени:
# Допустим, вы получаете в среднем 10 пятизвёздочных артефактов в день
# Тогда среднее ожидаемое время получить желаемый артефакт ~ 1/(p * 10 артеф/день)
if p > 0:
    artifacts_per_day = 10
    expected_days = 1 / (p * artifacts_per_day)
    print(f"Среднее ожидаемое время при {artifacts_per_day} артефактах в день считается как 1/(p * {artifacts_per_day}) = {expected_days} дней (с учетом перестановок)")
    monts = expected_days / 30
    print(f"Или {monts} месяцев")
    print(f"Или {monts / 12} лет")
else:
    print("Вероятность равна 0, такой артефакт невозможен по заданным условиям.")

Рассчёт вероятности получить артефакт с заданными параметрами:
Слот: Кубок, основной стат: ПИРО%, сабстаты: ['КШ', 'КУ', 'АТК%']

Вероятность слота: 0.2
Вероятность основного стата: 0.05
Вероятности сабстатов:
КШ: 3 / 44 = 0.0682, КУ: 3 / 41 = 0.0732, АТК%: 4 / 38 = 0.1053 => 0.0005251488
КШ: 3 / 44 = 0.0682, АТК%: 4 / 41 = 0.0976, КУ: 3 / 37 = 0.0811 => 0.0005393420
КУ: 3 / 44 = 0.0682, КШ: 3 / 41 = 0.0732, АТК%: 4 / 38 = 0.1053 => 0.0005251488
КУ: 3 / 44 = 0.0682, АТК%: 4 / 41 = 0.0976, КШ: 3 / 37 = 0.0811 => 0.0005393420
АТК%: 4 / 44 = 0.0909, КШ: 3 / 40 = 0.0750, КУ: 3 / 37 = 0.0811 => 0.0005528256
АТК%: 4 / 44 = 0.0909, КУ: 3 / 40 = 0.0750, КШ: 3 / 37 = 0.0811 => 0.0005528256
Суммарная вероятность сабстатов: 0.0032346326954799352

Вероятность получить Кубок с ПИРО% и сабстатами ['КШ', 'КУ', 'АТК%']:     0.000032
Среднее ожидаемое время при 10 артефактах в день считается как 1/(p * 10) = 3091.541124274779 дней (с учетом перестановок)
Или 103.05137080915931 месяцев
Или 8.58761423409

In [10]:
# Пример: хотим получить артефакт из слота кубок с основным статом "ПИРО%" и сабстатами ["КШ", "КУ", "АТК%"]
slot = "Цветок"
desired_main_stat = "HP"
desired_substats = ["КШ", "КУ", "HP%"]

print(f"Рассчёт вероятности получить артефакт с заданными параметрами:")
print(f"Слот: {slot}, основной стат: {desired_main_stat}, сабстаты: {desired_substats}\n")

print("Вероятность нужного сета артефактов: 0.5")
p = 0.5 * get_artifact_probability(slot, desired_main_stat, desired_substats)
print(f"Вероятность получить {slot} с {desired_main_stat} и сабстатами {desired_substats}: {p:12f}")

# Оценка времени:
# Допустим, вы получаете в среднем 10 пятизвёздочных артефактов в день
# Тогда среднее ожидаемое время получить желаемый артефакт ~ 1/(p * 10 артеф/день)
if p > 0:
    artifacts_per_day = 10
    expected_days = 1 / (p * artifacts_per_day)
    print(f"Среднее ожидаемое время при {artifacts_per_day} артефактах в день считается как 1/(p * {artifacts_per_day}) = {expected_days} дней (с учетом перестановок)")
    monts = expected_days / 30
    print(f"Или {monts} месяцев")
    print(f"Или {monts / 12} лет")
else:
    print("Вероятность равна 0, такой артефакт невозможен по заданным условиям.")

Рассчёт вероятности получить артефакт с заданными параметрами:
Слот: Цветок, основной стат: HP, сабстаты: ['КШ', 'КУ', 'HP%']

Вероятность нужного сета артефактов: 0.5
Вероятность слота: 0.2
Вероятность основного стата: 1.0
Вероятности сабстатов:
КШ: 3 / 38 = 0.0789, КУ: 3 / 35 = 0.0857, HP%: 4 / 32 = 0.1250 => 0.0008458647
КШ: 3 / 38 = 0.0789, HP%: 4 / 35 = 0.1143, КУ: 3 / 31 = 0.0968 => 0.0008731506
КУ: 3 / 38 = 0.0789, КШ: 3 / 35 = 0.0857, HP%: 4 / 32 = 0.1250 => 0.0008458647
КУ: 3 / 38 = 0.0789, HP%: 4 / 35 = 0.1143, КШ: 3 / 31 = 0.0968 => 0.0008731506
HP%: 4 / 38 = 0.1053, КШ: 3 / 34 = 0.0882, КУ: 3 / 31 = 0.0968 => 0.0008988315
HP%: 4 / 38 = 0.1053, КУ: 3 / 34 = 0.0882, КШ: 3 / 31 = 0.0968 => 0.0008988315
Суммарная вероятность сабстатов: 0.0052356935983221805

Вероятность получить Цветок с HP и сабстатами ['КШ', 'КУ', 'HP%']:     0.000524
Среднее ожидаемое время при 10 артефактах в день считается как 1/(p * 10) = 190.99666189794948 дней (с учетом перестановок)
Или 6.3665553965983

In [11]:
from itertools import permutations, combinations

def get_substats_set_probability(main_stat, desired_substats, total_substats_count):

    # Формируем пул сабстатов без основного
    available_substats = {k: v for k, v in extraStatsWeights.items()}
    if main_stat in available_substats:
        del available_substats[main_stat]

    # Проверим входные данные
    if len(desired_substats) > total_substats_count:
        print("Невозможно: желаемых сабстатов больше, чем всего требуемых.")
        return 0.0

    for s in desired_substats:
        if s not in available_substats:
            print(f"Сабстат {s} недоступен в пуле.")
            return 0.0

    d = len(desired_substats)
    needed = total_substats_count - d

    # Пул оставшихся сабстатов (которые не желаемые)
    remaining_pool = {k: v for k, v in available_substats.items() if k not in desired_substats}

    if needed == 0:
        # Если нам не нужно добавлять сабстаты сверх желаемых
        sets_to_check = [tuple(desired_substats)]
    else:
        # Генерируем все комбинации оставшихся сабстатов для заполнения
        all_combinations = combinations(remaining_pool.keys(), needed)
        sets_to_check = [tuple(desired_substats) + c for c in all_combinations]

    total_probability = 0.0

    print("Будем рассматривать следующие комбинации сабстатов:")
    for sc in sets_to_check:
        print("Комбинация:", sc)

    # Перебираем все выбранные комбинации и для каждой суммируем вероятность по всем перестановкам
    for full_set in sets_to_check:
        print(f"\nОбрабатываем комбинацию: {full_set}")
        # Для данной комбинации считаем сумму вероятностей по всем её перестановкам
        combination_probability = 0.0

        # Перестановки набора сабстатов:
        for perm in permutations(full_set):
            # Рассчёт вероятности для перестановки perm
            str = f"  Перестановка: {perm} => "
            pool = {k: v for k, v in available_substats.items()}

            p = 1.0
            for i in range(0, len(perm)):
                total_weight = sum(pool.values())
                w = pool[perm[i]]
                p_i = w / total_weight
                p *= p_i
                str += f" {perm[i]} {w} / {total_weight} = {p_i:.6f} "
                del pool[perm[i]]

            print(str + f" => {p:.6f}")
            combination_probability += p

        print(f"Вероятность для комбинации {full_set} (с учетом всех перестановок): {combination_probability:.6f}")
        total_probability += combination_probability

    return total_probability


In [12]:
def get_artifact_probability(slot, main_stat, desired_substats, total_substats_count):
    # Вероятность слота
    total_slot_weight = sum(slot_weights.values())
    p_slot = slot_weights[slot] / total_slot_weight
    print(f"Вероятность слота: {p_slot}")

    # Вероятность основного стата
    p_main = get_main_stat_probability(slot, main_stat)
    print(f"Вероятность основного стата: {p_main}")

    # Вероятность набора сабстатов (включая желаемые) для заданного количества
    p_sub = get_substats_set_probability(main_stat, desired_substats, total_substats_count)
    print(f"Суммарная вероятность сабстатов: {p_sub}\n")

    return p_slot * p_main * p_sub


In [13]:
# Пример: хотим получить артефакт из слота кубок с основным статом "ПИРО%" и сабстатами ["КШ", "КУ", "АТК%"]
slot = "Цветок"
desired_main_stat = "HP"
desired_substats = ["КШ", "КУ"]

print(f"Рассчёт вероятности получить артефакт с заданными параметрами:")
print(f"Слот: {slot}, основной стат: {desired_main_stat}, сабстаты: {desired_substats}\n")

p = get_artifact_probability(slot, desired_main_stat, desired_substats, 3)
print("Вероятность нужного сета артефактов: 0.5")
p *= 0.5
print(f"Вероятность получить {slot} с {desired_main_stat} и сабстатами {desired_substats}: {p:12f}")
# Оценка времени:
# Допустим, вы получаете в среднем 10 пятизвёздочных артефактов в день
# Тогда среднее ожидаемое время получить желаемый артефакт ~ 1/(p * 10 артеф/день)
if p > 0:
    artifacts_per_day = 10
    expected_days = 1 / (p * artifacts_per_day)
    print(f"Среднее ожидаемое время при {artifacts_per_day} артефактах в день считается как 1/(p * {artifacts_per_day}) = {expected_days} дней (с учетом перестановок)")
    monts = expected_days / 30
    print(f"Или {monts} месяцев")
else:
    print("Вероятность равна 0, такой артефакт невозможен по заданным условиям.")

Рассчёт вероятности получить артефакт с заданными параметрами:
Слот: Цветок, основной стат: HP, сабстаты: ['КШ', 'КУ']

Вероятность слота: 0.2
Вероятность основного стата: 1.0
Будем рассматривать следующие комбинации сабстатов:
Комбинация: ('КШ', 'КУ', 'HP%')
Комбинация: ('КШ', 'КУ', 'АТК')
Комбинация: ('КШ', 'КУ', 'АТК%')
Комбинация: ('КШ', 'КУ', 'ДЕФ')
Комбинация: ('КШ', 'КУ', 'ДЕФ%')
Комбинация: ('КШ', 'КУ', 'ВЭ%')
Комбинация: ('КШ', 'КУ', 'МС')

Обрабатываем комбинацию: ('КШ', 'КУ', 'HP%')
  Перестановка: ('КШ', 'КУ', 'HP%') =>  КШ 3 / 38 = 0.078947  КУ 3 / 35 = 0.085714  HP% 4 / 32 = 0.125000  => 0.000846
  Перестановка: ('КШ', 'HP%', 'КУ') =>  КШ 3 / 38 = 0.078947  HP% 4 / 35 = 0.114286  КУ 3 / 31 = 0.096774  => 0.000873
  Перестановка: ('КУ', 'КШ', 'HP%') =>  КУ 3 / 38 = 0.078947  КШ 3 / 35 = 0.085714  HP% 4 / 32 = 0.125000  => 0.000846
  Перестановка: ('КУ', 'HP%', 'КШ') =>  КУ 3 / 38 = 0.078947  HP% 4 / 35 = 0.114286  КШ 3 / 31 = 0.096774  => 0.000873
  Перестановка: ('HP%', '

In [14]:
# Пример: хотим получить артефакт из слота кубок с основным статом "ПИРО%" и сабстатами ["КШ", "КУ", "АТК%"]
slot = "Цветок"
desired_main_stat = "HP"
desired_substats = ["КШ", "КУ"]

print(f"Рассчёт вероятности получить артефакт с заданными параметрами:")
print(f"Слот: {slot}, основной стат: {desired_main_stat}, сабстаты: {desired_substats}\n")

p = get_artifact_probability(slot, desired_main_stat, desired_substats, 4)
print("Вероятность нужного сета артефактов: 0.5")
p *= 0.5
print(f"Вероятность получить {slot} с {desired_main_stat} и сабстатами {desired_substats}: {p:12f}")
# Оценка времени:
# Допустим, вы получаете в среднем 10 пятизвёздочных артефактов в день
# Тогда среднее ожидаемое время получить желаемый артефакт ~ 1/(p * 10 артеф/день)
if p > 0:
    artifacts_per_day = 10
    expected_days = 1 / (p * artifacts_per_day)
    print(f"Среднее ожидаемое время при {artifacts_per_day} артефактах в день считается как 1/(p * {artifacts_per_day}) = {expected_days} дней (с учетом перестановок)")
    monts = expected_days / 30
    print(f"Или {monts} месяцев")
else:
    print("Вероятность равна 0, такой артефакт невозможен по заданным условиям.")

Рассчёт вероятности получить артефакт с заданными параметрами:
Слот: Цветок, основной стат: HP, сабстаты: ['КШ', 'КУ']

Вероятность слота: 0.2
Вероятность основного стата: 1.0
Будем рассматривать следующие комбинации сабстатов:
Комбинация: ('КШ', 'КУ', 'HP%', 'АТК')
Комбинация: ('КШ', 'КУ', 'HP%', 'АТК%')
Комбинация: ('КШ', 'КУ', 'HP%', 'ДЕФ')
Комбинация: ('КШ', 'КУ', 'HP%', 'ДЕФ%')
Комбинация: ('КШ', 'КУ', 'HP%', 'ВЭ%')
Комбинация: ('КШ', 'КУ', 'HP%', 'МС')
Комбинация: ('КШ', 'КУ', 'АТК', 'АТК%')
Комбинация: ('КШ', 'КУ', 'АТК', 'ДЕФ')
Комбинация: ('КШ', 'КУ', 'АТК', 'ДЕФ%')
Комбинация: ('КШ', 'КУ', 'АТК', 'ВЭ%')
Комбинация: ('КШ', 'КУ', 'АТК', 'МС')
Комбинация: ('КШ', 'КУ', 'АТК%', 'ДЕФ')
Комбинация: ('КШ', 'КУ', 'АТК%', 'ДЕФ%')
Комбинация: ('КШ', 'КУ', 'АТК%', 'ВЭ%')
Комбинация: ('КШ', 'КУ', 'АТК%', 'МС')
Комбинация: ('КШ', 'КУ', 'ДЕФ', 'ДЕФ%')
Комбинация: ('КШ', 'КУ', 'ДЕФ', 'ВЭ%')
Комбинация: ('КШ', 'КУ', 'ДЕФ', 'МС')
Комбинация: ('КШ', 'КУ', 'ДЕФ%', 'ВЭ%')
Комбинация: ('КШ', 'КУ',

In [16]:
p_final = 0.009156*0.2 + 0.004298*0.8
print(p_final)
print("Среднее ожидаемое время при 10 артефактах в день считается как 1/(p * 10 артеф/день) = ", 1/p_final/10)

0.005269599999999999
Среднее ожидаемое время при 10 артефактах в день считается как 1/(p * 10 артеф/день) =  18.976772430545015


<div style="display: flex; gap: 10px;">
    <img src="stats3.png" alt="image1" width="600">
</div>